In [1]:
import glob
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal,stats
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.task import def_task,task_constants
from collections import defaultdict
from typing import Dict
import logging
from scipy import fft,signal

In [5]:
def indep_roll(arr, shifts, axis=1):
    """Apply an independent roll for each dimensions of a single axis.

    Parameters
    ----------
    arr : np.ndarray
        Array of any shape.

    shifts : np.ndarray
        How many shifting to use for each dimension. Shape: `(arr.shape[axis],)`.

    axis : int
        Axis along which elements are shifted. 
    """
    arr = np.swapaxes(arr,axis,-1)
    all_idcs = np.ogrid[[slice(0,n) for n in arr.shape]]

    # Convert to a positive shift
    shifts[shifts < 0] += arr.shape[-1] 
    all_idcs[-1] = all_idcs[-1] - shifts[:, np.newaxis]

    result = arr[tuple(all_idcs)]
    arr = np.swapaxes(result,-1,axis)
    return arr

In [7]:
sp.code_samples

array([[2327196., 2327220., 2327355., ...,      nan,      nan,      nan],
       [2331047., 2331056., 2331060., ...,      nan,      nan,      nan],
       [2334287., 2334303., 2334488., ...,      nan,      nan,      nan],
       ...,
       [6475950., 6475952., 6475965., ...,      nan,      nan,      nan],
       [6479044., 6479047., 6479059., ...,      nan,      nan,      nan],
       [6482703., 6482708., 6482718., ...,      nan,      nan,      nan]])

In [12]:
pp=np.array([np.arange(10),np.arange(10,20)])

In [13]:
pp

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])

In [15]:
indep_roll(pp, np.array([1,5]), axis=1)[:,2:5]

array([[ 1,  2,  3],
       [17, 18, 19]])

In [37]:
def get_fr_df(filepath,in_out,cgroup,e_align):
    py_f = np.load(filepath,allow_pickle=True).item(0)
    sp = py_f['sp_data']
    bhv = py_f['bhv']
    trial_idx = select_trials.select_trials_block(sp, n_block=1)
    trial_idx = select_trials.select_correct_trials(bhv, trial_idx)
    task = def_task.create_task_frame(trial_idx, bhv, task_constants.SAMPLES_COND)
    neurons = np.where(sp["clustersgroup"] == cgroup)[0]
    fr_samples = firing_rate.fr_by_sample_neuron(
            sp=sp,
            neurons=neurons,
            task=task,
            in_out=in_out,
            kernel=0,
            e_align=e_align,
            plot=False
        )
    return fr_samples

In [38]:
py_f = np.load(list[0],allow_pickle=True).item(0)
sp = py_f['sp_data']
bhv = py_f['bhv']

In [39]:
bhv[0].keys()

dict_keys(['trial', 'AbsoluteTrialStartTime', 'Btn1', 'Btn10', 'Btn2', 'Btn3', 'Btn4', 'Btn5', 'Btn6', 'Btn7', 'Btn8', 'Btn9', 'Eye', 'Eye2', 'EyeExtra', 'Gen1', 'Gen10', 'Gen11', 'Gen12', 'Gen13', 'Gen14', 'Gen15', 'Gen16', 'Gen2', 'Gen3', 'Gen4', 'Gen5', 'Gen6', 'Gen7', 'Gen8', 'Gen9', 'Joystick', 'Joystick2', 'KeyInput', 'Mouse', 'PhotoDiode', 'SampleInterval', 'Touch', 'CodeNumbers', 'CodeTimes', 'Block', 'BlockCount', 'Condition', 'CycleRate', 'Angle', 'BackgroundColor', 'Info', 'Position1', 'Position2', 'Position3', 'Position4', 'Position5', 'Position6', 'Scale', 'Status', 'Time', 'Zorder', 'ReactionTime', 'EndTimes', 'StartTimes', 'Attribute11', 'Attribute12', 'Attribute13', 'Attribute21', 'Attribute22', 'Attribute23', 'Attribute24', 'Attribute25', 'Attribute26', 'Code', 'FrameByFrameMovie', 'Size', 'Trial', 'TrialDateTime', 'TrialError', 'TrialWithinBlock', 'SkippedFrameTimeInfo', 'Dist', 'Eye2Offset', 'EyeOffset', 'Fix_FP_T_time', 'Fix_FP_post_T_time', 'Fix_FP_pre_T_time', 'Fi

In [40]:
# load all pfc files
pfc_path = "/home/INT/losada.c/Documents/data/test/areas/pfc/*"
list = glob.glob(pfc_path)
cgroup = "good"
in_out=1
e_align=2
fr_samples=[]
for path in list:
    print(path)
    fr_samples.append(get_fr_df(path,in_out,cgroup,e_align))


/home/INT/losada.c/Documents/data/test/areas/pfc/2022-11-22_10-59-03_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1308
Number of correct trials in block 2: 574
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-12-14_10-50-49_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1753
Number of correct trials in block 2: 874
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-11-30_10-42-03_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1588
Number of correct trials in block 2: 769
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-11-28_10-23-27_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1536
Number of correct trials in block 2: 657
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-12-16_10-30-59_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1340
Number of correct trials in block 2: 582


In [57]:
win_size = 100
step = 1
fix_duration = 100
rolling_df = fr_samples[1].loc[:,:'neuron'].iloc[:,:-1].rolling(window = win_size,axis=1,step=step,min_periods=1).mean()
rolling_df = rolling_df.iloc[:,fr_samples[1]['sample_on'][0]-200:]
rolling_df = pd.concat([rolling_df,fr_samples[1][['neuron','sample','trial_idx']]],axis=1)

In [59]:
rolling_df[rolling_df['trial_idx']==243]

,t1568,t1569,t1570,t1571,t1572,t1573,t1574,t1575,t1576,t1577,...,t5397,t5398,t5399,t5400,t5401,t5402,t5403,neuron,sample,trial_idx
0,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,1,o0_c0,243
510,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2,o0_c0,243
1020,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,3,o0_c0,243
1530,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,4,o0_c0,243
2040,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5,o0_c0,243
2550,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,6,o0_c0,243
3060,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7,o0_c0,243
3570,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8,o0_c0,243
4080,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.03,0.03,0.03,0.03,0.03,0.03,0.03,9,o0_c0,243
4590,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,10,o0_c0,243


In [50]:
fr_samples[1]

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,...,neuron,sample,trial_idx,start_trial,fixation,sample_on,sample_off,test_on_1,test_off_1,end_trial
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,o0_c0,243,1337,1562,1768,2227,2768,3227,5369
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,o0_c0,286,1549,1561,1768,2227,2686,3145,5287
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,o0_c0,313,1431,1557,1768,2227,2662,3121,5264
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,o0_c0,329,1507,1559,1768,2227,2686,3145,5287
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,o0_c0,346,1549,1560,1768,2227,2792,3251,5393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28,o5_c5,1945,1549,1560,1768,2227,2733,3039,3341
14276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28,o5_c5,1952,1354,1559,1768,2227,2663,3122,4742
14277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28,o5_c5,1959,1283,1563,1768,2227,2697,3156,5298
14278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,28,o5_c5,1971,1236,1555,1768,2226,2744,3062,3364


### SVM

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV,KFold,cross_val_score,StratifiedKFold
from sklearn.svm import SVC,LinearSVC
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics

In [ ]:
def run_svm_decoder(model,list_df,n_iterations,steps,win_steps,win_size,it_seed,resample_value):
    scores = np.zeros((n_iterations,steps))
    for n_it in np.arange(n_iterations):
        #  select trials randomly
        df = []
        for i_df in list_df:
            df.append(i_df.sample(resample_value,random_state=it_seed[n_it],replace=False)) # sample without replacement
        df = pd.concat(df).reset_index(drop=True)
        
        #df = df.sample(frac=1,random_state=it_seed[n_it],replace=False)
        
        X = df.iloc[:, :-4] 
        #X = pd.DataFrame(scaler.fit_transform(X))
        y = df['label_encoder']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,shuffle=True, random_state=it_seed[n_it])
        X_train['label_encoder'] = y_train
        #X_test['label_encoder'] = y_test
        # Sample with replacement
        X_train = X_train.sample(len(X_train),random_state=it_seed[n_it],replace=False)
        y_train = X_train['label_encoder']
        X_train = X_train.iloc[:,:-1]
        for n,n_win in enumerate(np.arange(0,steps*win_steps,win_steps)):
            # Split into X and y
            scaler = MinMaxScaler()
            X = X_train.iloc[:,int(n_win):int(n_win)+win_size].mean(axis=1).values.reshape(-1, 1)
            X = pd.DataFrame(scaler.fit_transform(X))    
            # test
            X_t = X_test.iloc[:,int(n_win):int(n_win)+win_size].mean(axis=1).values.reshape(-1, 1)        
            X_t = pd.DataFrame(scaler.transform(X_t))
            model.fit(X,y_train)
            y_predict = model.predict(X_t)
            scores[n_it, n] = metrics.accuracy_score(y_test,y_predict)# cross_val_score(model, X, y, cv=kf,n_jobs=-1).mean()
    return scores

In [ ]:
# plot results
def plot_accuracy(scores,win_steps,neuron_max_shift,x_lim_min,x_lim_max,n_neuron):
    fig,ax = plt.subplots()
    ax.plot(((np.arange(0,len(scores[0]))*win_steps)- neuron_max_shift[n_neuron-1])/1000,scores[:13].mean(axis=0))
    ax.set_xlim(x_lim_min, x_lim_max)
    ax.vlines(
        0, 0.3, 1, color="k", linestyles="dashed"
    )# sample on
    ax.hlines(
        0.5, x_lim_min, x_lim_max, color="gray", linestyles="solid"
    )
    ax.set_title('Is neuron %d engaged in the task?'%(n_neuron))
    ax.set(xlabel="Time (s)", ylabel="SVM classifier accuracy")
    fig.tight_layout(pad=0.2, h_pad=0.2, w_pad=0.2)
    fig.legend(['Accuracy','Sample on'], fontsize=9)